In [2]:
import urllib.request
import urllib.error
try:
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
    response = urllib.request.Request('http://python.org/',headers=headers)
    html = urllib.request.urlopen(response)
    result = html.read().decode('utf-8')
except urllib.error.URLError as e:
    if hasattr(e,'reeson'):
        print(e.reason)
except urllib.error.HTTPError as e:
    if hasattr(e,'code'):
        print(e.code)
else:
    print('great')
        

In [3]:
from collections import Counter
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=3000,n_features=2,n_informative=2,n_redundant=0,n_repeated=0,n_classes=3,n_clusters_per_class=1,weights=[0.1,0.05,0.85],class_sep=0.8,random_state=2018)
Counter(y)


Counter({2: 2532, 1: 163, 0: 305})

In [4]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled,y_resampled = ros.fit_sample(X,y)
sorted(Counter(y_resampled).items())

[(0, 2532), (1, 2532), (2, 2532)]

In [5]:
from imblearn.over_sampling import SMOTE
X_resampled_smote,y_resampled_smote = SMOTE().fit_sample(X,y)
sorted(Counter(y_resampled_smote).items())

[(0, 2532), (1, 2532), (2, 2532)]

In [7]:
from imblearn.over_sampling import ADASYN
X_resampled_adasyn,y_resampled_adasyn = ADASYN().fit_sample(X,y)
sorted(Counter(y_resampled_adasyn).items())

[(0, 2522), (1, 2520), (2, 2532)]

In [8]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled_under,y_resampled_under = rus.fit_sample(X,y)
sorted(Counter(y_resampled_under).items())

[(0, 163), (1, 163), (2, 163)]

In [11]:
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled_under_smote, y_resampled_under_smote = smote_enn.fit_sample(X,y)
sorted(Counter(y_resampled_under_smote).items())

[(0, 2111), (1, 2099), (2, 1893)]

In [12]:
from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=0)
X_resampled,y_resampled =smote_tomek.fit_sample(X,y)
sorted(Counter(y_resampled).items())

[(0, 2412), (1, 2414), (2, 2396)]

In [13]:
from sklearn.svm import SVC
svm_model = SVC(class_weight='balanced')
svm_model.fit(X,y)


SVC(class_weight='balanced')

In [17]:
from imblearn.ensemble import EasyEnsemble
ee = EasyEnsemble(random_state=0, n_subsets=10)
X_resampled, y_resampled = ee.fit_sample(X, y)
sorted(Counter(y_resampled[0]).items())

ImportError: cannot import name 'EasyEnsemble' from 'imblearn.ensemble' (/Users/yanshuo/opt/anaconda3/lib/python3.7/site-packages/imblearn/ensemble/__init__.py)

In [19]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
bbc = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),replacement=False,random_state=0)
bbc.fit(X,y)

BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                          random_state=0)

In [25]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
import numpy as np
from sklearn import datasets
X, y = datasets.make_classification(n_samples=10000,n_features=4,n_classes=2,weights=[0.95,0.05])
train_x,test_x,train_y,test_y = train_test_split(X,y,test_size=0.1,random_state=0)

In [30]:
rf = RandomForestClassifier(random_state=66,n_estimators=100)
from collections import Counter
rf.fit(train_x,train_y)
y_pred1 = rf.predict(test_x)
print(Counter(y_pred1),)
print('召回率',recall_score(test_y,y_pred1))


Counter({0: 963, 1: 37})
召回率 0.5185185185185185


In [31]:
from imblearn.ensemble import EasyEnsembleClassifier
ees = EasyEnsembleClassifier(random_state=66,sampling_strategy='not minority',n_estimators=100)
ees.fit(train_x,train_y)
y_pred = ees.predict(test_x)
print('召回率：',recall_score(test_y,y_pred))


召回率： 0.9074074074074074


In [33]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
X,y = make_classification(n_samples=10000,weights=[0.99],flip_y=0)
print(Counter(y))
oversample = RandomOverSampler(sampling_strategy='minority')
X_over,y_over = oversample.fit_resample(X,y)
print(Counter(y_over))

Counter({0: 9900, 1: 100})
Counter({0: 9900, 1: 9900})


In [40]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler


In [50]:
X,y = make_classification(n_samples=10000,weights=[0.99],flip_y=0)
steps = [('over', RandomOverSampler()), 
         ('model', DecisionTreeClassifier())]
pipeline = Pipeline(steps=steps)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(pipeline, X, y, scoring='f1_micro', cv=cv)
from numpy import mean
score = mean(scores)
print('F1 Score: %.3f' % score)

F1 Score: 0.984


In [51]:
from imblearn.under_sampling import RandomUnderSampler
over = RandomOverSampler(sampling_strategy=0.1)
X,y = over.fit_resample(X,y)
under = RandomUnderSampler(sampling_strategy=0.5)
X,y = under.fit_resample(X,y)

In [54]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
X,y =make_classification(n_samples=10000,weights=[0.99],flip_y=0)
print(Counter(y))
over = RandomOverSampler(sampling_strategy=0.1)
X,y = over.fit_resample(X,y)
print(Counter(y))
under = RandomUnderSampler(sampling_strategy=0.5)
X,y = under.fit_resample(X,y)
print(Counter(y))

Counter({0: 9900, 1: 100})
Counter({0: 9900, 1: 990})
Counter({0: 1980, 1: 990})


In [57]:

X, y = make_classification(n_samples=10000, weights=[0.99], flip_y=0)
over = RandomOverSampler(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under), ('m', DecisionTreeClassifier())]
pipeline = Pipeline(steps=steps)

cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)
scores = cross_val_score(pipeline,X,y,scoring='f1_micro',cv=cv)
score = mean(scores)
print(score)

0.9784333333333334


In [58]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import ADASYN
ros = RandomOverSampler(sampling_strategy={0:700,1:200,2:150},random_state=0)
X_resampled,y_resampled = ros.fit_sample(X,y)

smo = SMOTE(sampling_strategy={0:700,1:200,2:150},random_state=42)
X_smo,y_smo = smo.fit_sample(X,y)

bsmo = BorderlineSMOTE(kind='borderline-1',sampling_strategy={0:700,1:200,2:150},random_state=42)
X_smo,y_smo = bsmo.fit_sample(X,y)
ana = ADASYN(sampling_strategy={0:800,2:300,1:400},random_state=0)
x_ana,y_ana = ana.fit_sample(X,y)

ValueError: The {2} target class is/are not present in the data.

In [61]:
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import NearMiss

model_RandimUnderSmapler = RandomUnderSampler()
x_Rand_smapled ,y_Rand_sampled = model_RandimUnderSmapler.fit_sample(x,y)
import pandas as pd
x_Rand_smapled = pd.DataFrame(x_Rand_smapled,columns=[])
y_Rand_sampled = pd.DataFrame(y_Rand_sampled,columns=[])
data_sampled = pd.concat([x_Rand_smapled,y_Rand_sampled],axis=1)